In [2]:
import pandas as pd
import numpy as np
from math import sqrt

In [4]:
ratings_data = pd.read_csv('Downloads/ratings.csv')

In [6]:
ratings_data.head()

,user_id,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,0,2.0,3.0,4.0,NaN,2.0,3.0,NaN,NaN,NaN,...,4.0,NaN,NaN,1.0,NaN,NaN,2.0,5.0,2.0,NaN
1,1,NaN,NaN,NaN,4.0,NaN,5.0,NaN,NaN,2.0,...,4.0,NaN,1.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN
2,2,2.0,NaN,1.0,NaN,NaN,5.0,5.0,NaN,NaN,...,NaN,5.0,3.0,3.0,NaN,3.0,3.0,4.0,NaN,NaN
3,3,NaN,3.0,5.0,NaN,NaN,NaN,4.0,3.0,NaN,...,3.0,NaN,4.0,NaN,NaN,2.0,NaN,NaN,NaN,2.0
4,4,NaN,NaN,NaN,5.0,NaN,NaN,2.0,2.0,2.0,...,3.0,4.0,2.0,5.0,4.0,NaN,NaN,NaN,4.0,2.0


In [9]:
np.set_printoptions(precision=2)  # 소수점 둘째 자리까지만 출력

In [17]:
def distance(user_1, user_2):
    """유클리드 거리를 계산해주는 함수"""
    return sqrt(np.sum(user_1-user_2)**2) 
    
def filter_users_without_movie(ratings_data, movie_id):
    """movie_id 번째 영화를 평가하지 않은 유저들은 미리 제외해주는 함수"""
    return ratings_data[~np.isnan(ratings_data[:,movie_id])]

def fill_nan_with_user_mean(ratings_data):
    """평점 데이터의 빈값들을 각 유저 평균 값으로 채워주는 함수"""
    filled_data = np.copy(ratings_data)  # 평점 데이터를 훼손하지 않기 위해 복사
    row_mean = np.nanmean(filled_data, axis=0)  # 유저 평균 평점 계산
    
    index = np.where(np.isnan(filled_data))  # 비어 있는 인덱스들을 구한다
    filled_data[index] = np.take(row_mean, index[1])  #빈 인덱스를 유저 평점으로 채운다
    
    return filled_data

def get_k_neighbors(user_id, rating_data, k):
    """user_id에 해당하는 유저의 이웃들을 찾아주는 함수"""
    distance_data = np.copy(rating_data)  # 평점 데이터를 훼손하지 않기 위해 복사
    # 마지막에 거리 데이터를 담을 열 추가한다
    distance_data = np.append(distance_data, np.zeros((distance_data.shape[0], 1)), axis=1)
    
    # 코드를 쓰세요.
    for i in range(len(distance_data)):
        row = distance_data[i]
        
        if i == user_id:  # 같은 유저면 거리를 무한대로 설정
            row[-1] = np.inf
        else:  # 다른 유저면 마지막 열에 거리 데이터를 저장
            row[-1] = distance(distance_data[user_id][:-1], row[:-1])
    
    # 데이터를 거리 열을 기준으로 정렬한다
    distance_data = distance_data[np.argsort(distance_data[:, -1])]
    
    # 가장 가까운 k개의 행만 리턴한다 + 마지막(거리) 열은 제외한다
    return distance_data[:k, :-1]
    
def predict_user_rating(ratings_data, k, user_id, movie_id,):
    """예측 행렬에 따라 유저의 영화 평점 예측 값 구하기"""
    # movie_id 번째 영화를 보지 않은 유저를 데이터에서 미리 제외시킨다
    filtered_data = filter_users_without_movie(ratings_data, movie_id)
    # 빈값들이 채워진 새로운 행렬을 만든다
    filled_data = fill_nan_with_user_mean(filtered_data)
    # 유저 user_id와 비슷한 k개의 유저 데이터를 찾는다
    neighbors = get_k_neighbors(user_id, filled_data, k)
    
    # 코드를 쓰세요
    return np.mean(neighbors[:, movie_id])
